In [1]:
import os
from firebase_admin import credentials, firestore, initialize_app

# Ref: https://firebase.google.com/docs/firestore/query-data/get-data
# Ref: https://firebase.google.com/docs/firestore/query-data/queries *** NEXT: FOR PROJECT WORK ***
# Querying Cloud Store to find documents in a collection by search criteria

In [2]:
cred = credentials.Certificate('shiva.json')
default_app = initialize_app(cred) # Do we need it? Yes!
db = firestore.client()

In [15]:
# Create a class in ORM-style.
# To serialize an object for database operations, 
# 1. Implement __repr__ to render attributes in a serializable format (dict/json)
# 2. Use vars() for identical result

class City(object):
    def __init__(self, name, state, country, capital=False, population=0,
                 regions=[]):
        self.name = name
        self.state = state
        self.country = country
        self.capital = capital
        self.population = population
        self.regions = regions
        

    @staticmethod
    def from_dict(source):
        # ...
        pass
    
    def to_dict(self):
        # ...
        pass

    def __repr__(self):
        return(
            u'City(name={}, state={}, country={}, capital={}, population={}, regions={})'
            .format(self.name, self.state, self.country, self.capital, self.population, self.regions))
    
# Ref: https://stackoverflow.com/questions/61517/python-dictionary-from-an-objects-fields

In [46]:
# Initialize an object and introspect by serialization
# Serialization is required to render an object as a json string
# for insertion into firestore database
SF = City(u'San Francisco', u'CA', u'USA', False, 860000,
         [u'west_coast', u'norcal'])
print("Serialized to a dict as {}".format(SF.__dict__))
print("Serialized to a dict as {}".format(vars(SF)))

Serialized to a dict as {'name': 'San Francisco', 'state': 'CA', 'country': 'USA', 'capital': False, 'population': 860000, 'regions': ['west_coast', 'norcal']}
Serialized to a dict as {'name': 'San Francisco', 'state': 'CA', 'country': 'USA', 'capital': False, 'population': 860000, 'regions': ['west_coast', 'norcal']}


In [47]:
# (C)RUD - Insert new data in a document
# Note: Serialization with __dict__ enables inserting objects in database as dict/json
# Optionally, use vars() which accepts the object as an argument.
cities_ref = db.collection(u'cities')
cities_ref.document(u'SF').set(
    City(u'San Francisco', u'CA', u'USA', False, 860000,
         [u'west_coast', u'norcal']).__dict__)
cities_ref.document(u'LA').set(
    City(u'Los Angeles', u'CA', u'USA', False, 3900000,
         [u'west_coast', u'socal']).__dict__)
cities_ref.document(u'DC').set(
    City(u'Washington D.C.', None, u'USA', True, 680000,
         [u'east_coast']).__dict__)
cities_ref.document(u'TOK').set(
    City(u'Tokyo', None, u'Japan', True, 9000000,
         [u'kanto', u'honshu']).__dict__)
cities_ref.document(u'BJ').set(
    City(u'Beijing', None, u'China', True, 21500000, [u'hebei']).__dict__)

update_time {
  seconds: 1573524496
  nanos: 537662000
}

In [67]:
# C(R)UD - Retrieve all documents in store

# Get all cities in California
cities_ref = db.collection(u'cities')
cities_query = cities_ref.where(u'state', u'==', u'CA')
for city in cities_query.stream():
    print(u"Retrieved CA cities with id {} and details {}.".format(city.id, city.to_dict()))

# Get all cities that are national capitals
capitals_query = cities_ref.where(u'capital', u'==', True)
for capital in capitals_query.stream():
    print(u"Retrieved capital with id {},name {} and population {}".format(capital.id, 
                                                                           capital.to_dict()[u"name"],
                                                                           capital.to_dict()[u"population"]))
# Get all cities with a population in excess of 1M
pop_query = cities_ref.where(u'population', u'>', 1000000)
for populous_city in pop_query.stream():
    print(u'Retrieved populous city with id {} in {} with population {}'.format(populous_city.id,
                                                                               populous_city.to_dict()[u"country"],
                                                                               populous_city.to_dict()[u"population"]))
    
# Get cities on the west coast
coast_query = cities_ref.where(u'regions', u'array_contains', u'west_coast')
for coastal_city in coast_query.stream():
    print("See the sunset at {} in {}".format(coastal_city.id, coastal_city.to_dict()[u'state']))
    
# Get the populous capital cities
# Requires indexing on the compound key,
# if none exists, one will be suggested
popCapital_query = cities_ref.where(u'population', u'>=', 1000000).where(u'capital', u'==', True)
for populous_capital in popCapital_query.stream():
    print("{} is a populous capital city in {}".format(populous_capital.to_dict()[u'name'], 
                                                      populous_capital.to_dict()[u'country']))

Retrieved CA cities with id LA and details {'name': 'Los Angeles', 'population': 3900000, 'regions': ['west_coast', 'socal'], 'country': 'USA', 'state': 'CA', 'capital': False}.
Retrieved CA cities with id SF and details {'name': 'San Francisco', 'regions': ['west_coast', 'norcal'], 'population': 860000, 'country': 'USA', 'state': 'CA', 'capital': False}.
Retrieved capital with id BJ,name Beijing and population 21500000
Retrieved capital with id DC,name Washington D.C. and population 680000
Retrieved capital with id TOK,name Tokyo and population 9000000
Retrieved populous city with id LA in USA with population 3900000
Retrieved populous city with id TOK in Japan with population 9000000
Retrieved populous city with id BJ in China with population 21500000
See the sunset at LA in CA
See the sunset at SF in CA
Tokyo is a populous capital city in Japan
Beijing is a populous capital city in China
